In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%load_ext autoreload
%autoreload 2from fastai.imports import *
from fastai.structured import *
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary
import sklearn.model_selection
from IPython.display import display
import math
import random
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import collections
%matplotlib inline

Required Functions

In [ ]:
def imae(x,y):
    return 1/(1+(abs(x-y)).mean())

In [ ]:
def print_score(m):
    res = [
        imae(m.predict(X_train.drop(['Unnamed: 0'],axis=1)), y_train),
        imae(m.predict(X_valid.drop(['Unnamed: 0'],axis=1)), y_valid),
        m.score(X_train.drop(['Unnamed: 0'],axis=1), y_train), 
        m.score(X_valid.drop(['Unnamed: 0'],axis=1), y_valid),
          ]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000):
        display(df)

Data PreProcessing

In [ ]:
df_i = pd.read_csv(f'{PATH}sample_submission.csv')
df_i.shot_id_number = df_i.shot_id_number-1
df_i=df_i.drop(['is_goal'], axis=1)

In [ ]:
df_raw = pd.read_csv(f'{PATH}data.csv', low_memory=False, parse_dates=['date_of_game'])

In [ ]:
df_raw.is_goal.value_counts()

In [ ]:
display_all(df_raw.T)

In [ ]:
df_raw['date_of_game'] = pd.to_datetime(df_raw.date_of_game)
df_raw=df_raw.sort_values('date_of_game')
display_all(df_raw.T)

In [ ]:
train_cats(df_raw)
cols = ['knockout_match','match_event_id', 'game_season', 'area_of_shot','shot_basics', 'range_of_shot', 'lat/lng', 'team_name',
       'home/away', 'shot_id_number',  'type_of_shot', 'type_of_combined_shot','match_id','team_id']
for col in cols:
    df_raw[col] = df_raw[col].astype('category').cat.codes
df_raw['year'] = df_raw['date_of_game'].dt.year
df_raw['month']=df_raw['date_of_game'].dt.month
df_raw=df_raw.drop(['date_of_game'],axis=1)
df_raw, __, ___ = proc_df(df_raw)

In [ ]:
df_raw.match_event_id.value_counts()

In [ ]:
df_tst = df_raw[df_raw['Unnamed: 0'].isin(df_i['shot_id_number'])]
df_trn = df_raw[~df_raw['Unnamed: 0'].isin(df_i['shot_id_number'])]
display_all(df_trn.T)

In [ ]:
X_train, X_valid, y_train, y_valid  = sklearn.model_selection.train_test_split(df_trn.drop(['is_goal'],axis=1), df_trn['is_goal'], test_size=0.20, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

Model Selection and Analysis

In [ ]:
set_rf_samples(100)
clf = RandomForestClassifier(n_estimators=1000, max_depth=10, max_features=0.5, min_samples_leaf=5)
clf.fit(X_train.drop(['Unnamed: 0'],axis=1), y_train)
print_score(clf)
[0.7244502396391317, 0.7206954570947841, 0.6196429440093398, 0.6124513618677043]
draw_tree(clf.estimators_[0], X_train.drop(['Unnamed: 0'],axis=1), precision=3)

In [ ]:
pred_valid =  clf.predict(X_train.drop(['Unnamed: 0'],axis=1))
collections.Counter(pred_valid)
Counter({0.0: 15933, 1.0: 4624})
In [52]:
preds = np.stack([t.predict(X_valid.drop(['Unnamed: 0'],axis=1)) for t in clf.estimators_])
#preds[:,0], np.mean(preds[:,0]), y_valid[0]
In [65]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(100)]);

In [ ]:
fi = rf_feat_importance(clf, X_valid.drop(['Unnamed: 0'],axis=1))
fi[:]

In [ ]:
clf.fit(df_trn.drop(['Unnamed: 0','is_goal'],axis=1), df_trn['is_goal'])
df_tst = df_tst.drop(['is_goal'],axis=1)
df_trn.T

Final Model

In [ ]:
pred_tst = clf.predict(df_tst.drop(['Unnamed: 0'],axis=1))
pred_tst
array([0., 1., 0., ..., 0., 0., 1.])
df_ans=pd.DataFrame()
df_ans['id']=df_tst['Unnamed: 0']+1
df_ans['prediction']=pred_tst.astype('int')
df_ans=df_ans.sort_values('id')
#df_ans.prediction = df_ans.prediction.astype('int')
df_ans.to_csv(f'{PATH}/amit_dubey_190199_code_5.csv', index=False
df_ans.T